In [1]:
import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto

# Create one input (ValueInfoProto)
X1 = helper.make_tensor_value_info("X1", TensorProto.FLOAT, [3, 2])
X2 = helper.make_tensor_value_info("X2", TensorProto.FLOAT, [3, 2])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [3, 2])

# Create a node (NodeProto) - This is based on Pad-11
node_def = helper.make_node(
    "Add",  # node name
    ["X1", "X2"],  # inputs
    ["Y"],  # outputs
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    "test-model",
    [X1, X2],
    [Y],
)

# Create the model (ModelProto)
model_def = helper.make_model(graph_def, producer_name="onnx-example")

print("The model is:\n{}".format(model_def))
onnx.checker.check_model(model_def)
onnx.save(model_def, "/home/yssun/onnx_mlir/add.onnx")
print("The model is checked!")

The model is:
ir_version: 10
opset_import {
  version: 21
}
producer_name: "onnx-example"
graph {
  node {
    input: "X1"
    input: "X2"
    output: "Y"
    op_type: "Add"
  }
  name: "test-model"
  input {
    name: "X1"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
  input {
    name: "X2"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
  output {
    name: "Y"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
}

The model is checked!


In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
import torch.fx as fx

In [3]:
class TestModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.mlp = torch.nn.Linear(input_dim, output_dim)
    
    
  def forward(self,x):
    return self.mlp(x)

In [4]:
model = TestModel(100,300)

In [6]:
import torch.fx as fx
traced = fx.symbolic_trace(model)

In [7]:
traced.code

'\n\n\ndef forward(self, x):\n    mlp = self.mlp(x);  x = None\n    return mlp\n    '

In [8]:
s = """
def test_func(x):
  return x + 1
"""

In [9]:
ori_code = traced.code

In [10]:
new_modify_code =s +  ori_code
print(new_modify_code)


def test_func(x):
  return x + 1



def forward(self, x):
    mlp = self.mlp(x);  x = None
    return mlp
    


In [12]:
exec(new_modify_code)

In [21]:
new_modify_code_list = new_modify_code.split("\n")
new_modify_code_list

['',
 'def test_func(x):',
 '  return x + 1',
 '',
 '',
 '',
 'def forward(self, x):',
 '    mlp = self.mlp(x);  x = None',
 '    return mlp',
 '    ']

In [22]:
new_modify_code_list.insert(7,"    y=test_func(x)")

In [23]:
new_modify_code_list

['',
 'def test_func(x):',
 '  return x + 1',
 '',
 '',
 '',
 'def forward(self, x):',
 '    y=test_func(x)',
 '    mlp = self.mlp(x);  x = None',
 '    return mlp',
 '    ']

In [26]:
new_modify_code_after = '\n'.join(new_modify_code_list)

In [27]:
print(new_modify_code_after)


def test_func(x):
  return x + 1



def forward(self, x):
    y=test_func(x)
    mlp = self.mlp(x);  x = None
    return mlp
    


In [28]:
exec(new_modify_code_after)

In [29]:
traced = fx.symbolic_trace(forward)

In [31]:
print(traced.code)




def forward(self, x):
    add = x + 1
    mlp = self.mlp(x);  self = x = None
    return mlp
    
